In [1]:
import numpy as np

In [ ]:
def compute_curvature(contour,i, window_size=5):
    """
    Compute the curvature at a specific point on the contour.

    Args:
        i (int): Index of the point on the contour.
        window_size (int): Number of points in the neighborhood for fitting the polynomial.

    Returns:
        float: The curvature at the specified point.
    """

    # Handle edge cases
    if len(contour) < 3 or i < 0 or i >= len(contour):
        raise ValueError("Invalid index or contour is too small.")

    # Extract neighborhood points
    # start = max(0, i - window_size // 2)
    # end = min(len(contour), i + window_size // 2 + 1)
    start = i - window_size // 2
    end = i + window_size // 2 + 1  # Inclusive of center, exclusive of end

    # Handle circular indexing using numpy
    indices = np.arange(start, end) % len(contour)

    neighborhood = contour[indices]
    x_neighborhood = neighborhood[:, 1]
    y_neighborhood = neighborhood[:, 0]

    # Compute tangent direction for the central point
    point = contour[i]
    tangent_direction = np.arctan2(
        y_neighborhood[-1] - y_neighborhood[0],
        x_neighborhood[-1] - x_neighborhood[0]
    )

    # Translate and rotate neighborhood points
    translated_x = x_neighborhood - point[1]
    translated_y = y_neighborhood - point[0]
    rotated_x = translated_x * np.cos(-tangent_direction) - translated_y * np.sin(-tangent_direction)
    rotated_y = translated_x * np.sin(-tangent_direction) + translated_y * np.cos(-tangent_direction)

    # Fit a polynomial of degree 2 to the rotated points
    coeffs = np.polyfit(rotated_x, rotated_y, 3)
    # Compute first and second derivatives of the fitted polynomial
    p = np.poly1d(coeffs)
    p_prime = p.deriv()
    p_double_prime = p_prime.deriv()

    # Compute curvature at a specific x-point (assuming x0 is given)
    
    y_prime = p_prime(0)
    y_double_prime = p_double_prime(0)

    # Compute curvature using the standard formula
    curvature = y_double_prime / (1 + y_prime**2) ** (3/2)
    return curvature*100

In [ ]:
def compute_curvature_profile(contour, min_contour_length=20, window_size_ratio=5):
    """
    Compute the curvature profile for all points on the contour.

    Args:
        min_contour_length (int): Minimum contour length to process.
        window_size_ratio (int): Ratio to determine the window size for curvature computation.

    Returns:
        None: Stores the curvature profile internally.
    """
    if len(contour) < min_contour_length:
        raise ValueError("Contour length is too short for curvature computation.")

    # Prepare for storing curvature values
    curvature_values = []
    edge_pixels = []

    # Iterate over each point in the contour
    contour = contour.squeeze()  # Remove redundant dimensions
    window_size = int(len(contour) / window_size_ratio)

    for j, point in enumerate(contour):
        curvature = compute_curvature(contour, j, window_size=window_size)
        curvature_values.append(curvature)
        edge_pixels.append(point)

    # Store the results
    smoothed_arr = gaussian_filter1d(curvature_values, sigma=15, mode='wrap')


    curvature_profile = {
        "original_curvature_values": np.array(curvature_values),
        'smooth_curvature_values': np.array(smoothed_arr)
    }

    return curvature_profile